빅데이터 분석 기말고사

각 문제 아래에 답을 쓰고 실행하여 제출하세요. 
반드시 3시전까지 업로드하세요.

1. 선형회귀에 대해서 설명하세요. 선형회귀를 경사하강법으로 해결할 때, 손실함수를 어떻게 가정하는지 설명하세요. (10pt)

답)

A) 선형회귀는 데이터의 피처와 타겟 변수 사이의 관계를 선형으로 모델링하여, 해당 선형모델링의 계수를 찾음으로써, 그 관계를 찾는 것을 의미한다. 선형회귀를 경사하강법으로 해결할 때 손실함수는 선형모델을 통한 출력값과 실제 타겟 변수 사이의 차이를 바탕으로 가정하는데, 본 수업에서는 제곱합으로 손실함수를 가정하였다.

2. advertising1,2는 가정별 TV,신문,라디오를 이용하는 시간과 Sales 수치를 나타내서, 어떤 매체를 통해 광고하는 것이 효율적인 지를 분석할 수 있는 데이터입니다. 아래 스텝을 따라서 분석하세요. 타겟 변수는 Sales입니다. 

2. (a) advertising1.csv와 advertising2.csv를 판다스의 read_csv를 통해서 load하여  advertising1, advertising2의 이름으로 할당하세요. 또한 두 데이터를 advertising의 이름으로 병합하세요, 병합 시에 ignore_index 옵션을 활용하여, index가 0~199까지 되게 하세요. (5pt)

답)

In [43]:
import pandas as pd
advertising1=pd.read_csv("advertising/advertising1.csv")
advertising2=pd.read_csv("advertising/advertising2.csv")
advertising=pd.concat([advertising1,advertising2],ignore_index=True)
advertising

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,14.0
197,177.0,9.3,6.4,14.8
198,283.6,42.0,66.2,25.5


2. (b) advertising 데이터를 훈련 데이터, 검증 데이터, 테스트 데이터로 분할하되 3:1:1 의 비율로 분할 하세요. (random_state는 1234로 하세요.) (5pt)

답)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(advertising.drop(columns='Sales'), 
                                                    advertising.Sales,
                                                    test_size=0.2, random_state=1234)
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                    y_train,
                                                    test_size=0.25, random_state=1234)

2. (c) 훈련데이터의 분포를 바탕으로, 훈련데이터, 검증데이터, 테스트데이터에 표준 정규화를 적용하세요. (5pt)

답)

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_val = scaler.transform(X_val)  
X_test = scaler.transform(X_test)  

2. (d) 준비된 데이터 셋에 대해 SGDRegressor를 활용하여 경사하강법을 통한 선형회귀를 수행하고자 합니다. 이 때 아래의 학습률 후보군 및 학습률 스케쥴링 전략 중에서 MAE가 가장 낮은 학습률 및 스케쥴링 전략을 찾아서 프린트하세요. 또한 해당 학습률과 스케쥴링 전략으로 훈련시켰을 때 각 피처별 계수를 찾으세요. (절편, 정규화는 사용하지 않으며 최대 반복수는 10000으로 설정합니다. 또한 random_state는 1234로 하세요. GridSearchCV는 사용하지 않습니다.) (10pt)

In [46]:
list_rate=[1e-4,1e-3,1e-2,1e-1]
list_scheduling=['constant','invscaling']

답)

In [47]:
from sklearn.linear_model import SGDRegressor
import numpy as np 
best_MAE=1000
best_rate=0
best_scheduling='constant'
for eta0 in list_rate:
    for learning_rate in list_scheduling:
        reg = SGDRegressor(fit_intercept=False, penalty=None, max_iter=10000, learning_rate=learning_rate, eta0=eta0, random_state=1234)
        reg = reg.fit(X_train, y_train)
        y_pred = reg.predict(X_val)
        if best_MAE > np.abs(y_pred - y_val).mean():
            best_rate=eta0
            best_scheduling=learning_rate

reg = SGDRegressor(fit_intercept=False, penalty=None, max_iter=10000, learning_rate=best_scheduling, eta0=best_rate, random_state=1234)
reg = reg.fit(X_train, y_train)
print(best_rate, best_scheduling)
print(reg.coef_)

0.1 invscaling
[4.37678034 0.44568746 0.04916637]


3. 머신러닝 모델의 과대적합과 과소적합에 대해 설명하고, 이와 연관지어서 라쏘회귀와 릿지회귀에 대해 설명하세요. (10pt)

답) 과대적합은 학습데이터의 수에 비해 모델이 복잡한 경우를 포함하여, 학습데이터에 대해 과하게 학습되는 경우를 의미한다. 이로 인해 그 결과는 편향이 낮고 분산이 크며, 일반화 능력이 떨어진다. 과소적합은 그 반대로, 모델이 너무 단순하거나 학습이 충분히 되지 않아서 학습데이터에 대해 잘 학습하지 못하는 경우를 의미한다. 이에 따라 편향이 크고 분산이 낮다. 모델 학습의 대표적인 문제는 과대적합이고, 라쏘회귀와 릿지회귀는 이를 해결하기 위해 선형회귀에서 정규화 항을 추가한다. 해당 정규화 항은 모델의 계수의 크기 및 계수 별 분산을 낮추고자 하며 L1 놈 기반의 정규화항인 경우에 라쏘회귀, L2 놈 기반의 정규화 항인 경우에는 릿지회귀라고 한다. 

4. 타이타닉 데이터는 각 승객별 여러가지 피처와 생존 여부 (타깃변수)를 나타내는 데이터입니다. 함께 첨부한 titanic1.csv~titanic4.csv 파일을 바탕으로 로지스틱회귀를 하고자 합니다. 아래 문제를 해결하세요.  

4. (a) 데이터 준비 및 병합을 수행하세요. 각 파일에서 로드한 데이터프레임을 잘 관찰하여, 최종적으로 891개의 데이터에 대해 12개의 피처(타변수 포함)를 갖도록 병합하세요. (5pt)

답)

In [21]:
import pandas as pd
titanic1=pd.read_csv("titanic/titanic1.csv")
titanic2=pd.read_csv("titanic/titanic2.csv")
titanic3=pd.read_csv("titanic/titanic3.csv")
titanic4=pd.read_csv("titanic/titanic4.csv")

titanic_12=pd.concat([titanic1,titanic2],ignore_index=True)
titanic_34=pd.concat([titanic3,titanic4],ignore_index=True)
titanic=pd.merge(titanic_12,titanic_34,on="PassengerId")

4. (b) 데이터 프레임에서 Pclass, Sex, Age, SibSp, Embarked만을 남겨서 X로, Survived만을 y로 따로 저장하세요. 또한 Sex 피처에서 male은 1, female은 0으로, Embarked에서 S는 0, C는 1, Q는 2로 변환하세요. (5pt)

답)

In [22]:
X=titanic.reindex(['Pclass','Sex','Age','SibSp','Embarked'],axis=1)
y=titanic.Survived
X['Sex']=X['Sex'].map({'male':1,"female":0})
X['Embarked']=X['Embarked'].map({'S':0,'C':1, 'Q':2})

4. (c) 훈련 데이터와 테스트 데이터를 3:1의 비율로 분할하세요. (random_state는 1234로 하세요.) (5pt) 

답)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.25, random_state=1234)

4. (d) GridSearchCV를 통해, 로지스틱회귀 수행시에 최적의 정규화계수 (C)를 튜닝하고자 합니다. 아래 후보군 주에서 최적의 정규화계수를 찾아서 출력하세요. (결측치처리 객체, 표준정규화 객체를 로지스틱회귀 전에 수행해야 하며, 4-fold 교차검증을 활용합니다.) (로지스틱회귀는 L2정규화를 수행하고 최대 반복수는 10000, solver는 'sag'로 하세요.) (k-fold의 random_state는 1234로 하세요.) (10pt)

In [24]:
l_C=[0.0001, 0.001, 0.01, 1, 10, 100, 1000]

답)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',StandardScaler()),('basemodel',LogisticRegression( solver='sag', penalty='l2', max_iter=10000))])
param_grid={'basemodel__C':l_C}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234)  
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 정규화 계수:',grid_search.best_params_['basemodel__C'])

최적 정규화 계수: 0.01


4. (e) 학습된 모델에 대해 테스트데이터 셋으로 실험하였을 때, 생존에 대한 재현율은 몇입니까? (코드를 작성하고 출력된 결과를 보고 답하세요.) (5pt)

답) 0.62

In [ ]:
from sklearn.metrics import classification_report
y_pred = grid_search.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.95      0.86       132
           1       0.89      0.62      0.73        91

    accuracy                           0.81       223
   macro avg       0.84      0.78      0.79       223
weighted avg       0.83      0.81      0.80       223



5. 의사결정트리에 관해서 설명하세요. 구체적으로, (a) 의사결정트리의 노드의 정의, (b) 불순도를 통한 학습원리의 설명, (c) 태스크가 분류/회귀 일때 테스트 데이터에 대한 예측 방법을 설명하세요. (10pt) (각각 3pt, 다 맞으면 10pt)

답) a) 의사결정트리는 각 노드들은 분할 속성을 바탕으로 한 분할 기준이다. (b) 여러 학습방법이 있지만, 기본적으로 노드를 통해 분할된 그룹의 불순도가 낮아지도록 한다. (c) 태스크가 분류일 때는 리프노드에 속한 데이터들의 타깃변수 중 가장 많은 레이블, 회귀일때는 평균으로 예측. 

6. 보스턴 집값 데이터는 집값과 관련된 여러가지 피처와, 보스턴의 집값을 나타내는 데이터입니다. 보스턴 집값 데이터 (boston.csv)를 분석하고 하는데, 여러 모델들에 대해 각각 하이퍼 파라미터 튜닝 및 성능 비교를 하고자 합니다. (타깃 변수는 MEDV 입니다.) 아래 질문에 대해 답하세요.

6. (a) 데이터 셋을 로드하고, 학습 데이터와 테스트 데이터 셋을 3:1의 비율로 분할 하세요. random_state는 1234로 하세요. (5pt)

답)

In [53]:
from sklearn.model_selection import train_test_split
boston=pd.read_csv("boston/boston.csv")
X_train, X_test, y_train, y_test = train_test_split(boston.drop(columns="MEDV"), 
                                                    boston.MEDV,
                                                    test_size=0.25, random_state=1234)

6. (b) 선형회귀, 라쏘회귀, 릿지회귀 기법을 통해, boston 데이터셋에 대해 최적의 하이퍼파라미터를 튜닝하고자 합니다. 선형회귀는 아래 리스트 중 l_linear로 시작하는 리스트를 후보군으로, 학습율과 학습율 스케쥴링 방법을 튜닝합니다. 라쏘회귀와 릿지회귀는 l_lasso, l_ridge로 시작하는 리스트를 후보군으로 학습율과 정규화 계수를 튜닝합니다. GridSearchCV를 활용하되, 결측치처리 객체, 최소최대정규화 객체를 회귀 전에 수행해야 하며, 4-fold 교차검증을 활용합니다. (최대 반복수는 10000으로 하고 k-fold 및 회귀객체의 random_state는 1234로 하세요.) 각 기법별로 최적의 하이퍼파라미터를 출력하세요. (10pt)

In [54]:
l_linear_rate=[1e-4,1e-3,1e-2,1e-1]
l_linear_scheduling=['constant','invscaling']
l_lasso_rate=[1e-4,1e-3,1e-2,1e-1]
l_lasso_reg=[1e-4,1e-3,1e-2,1e-1]
l_ridge_rate=[1e-4,1e-3,1e-2,1e-1]
l_ridge_reg=[1e-4,1e-3,1e-2,1e-1]

In [55]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor

R2_scores=[]

pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',SGDRegressor(penalty=None, max_iter=10000, random_state=1234))])
param_grid={'basemodel__eta0':l_linear_rate,'basemodel__learning_rate':l_linear_scheduling}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234)  
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 선형회귀 학습율:',grid_search.best_params_['basemodel__eta0'])
print('최적 선형회귀 스케쥴링:',grid_search.best_params_['basemodel__learning_rate'])
R2_scores.append(grid_search.score(X_test,y_test))




pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',SGDRegressor(penalty='l1', max_iter=10000, random_state=1234))])
param_grid={'basemodel__eta0':l_lasso_rate,'basemodel__alpha':l_lasso_reg}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234)  
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 라쏘 학습율:',grid_search.best_params_['basemodel__eta0'])
print('최적 라쏘 정규화계수:',grid_search.best_params_['basemodel__alpha'])
R2_scores.append(grid_search.score(X_test,y_test))


pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',SGDRegressor(penalty='l2', max_iter=10000, random_state=1234))])
param_grid={'basemodel__eta0':l_ridge_rate,'basemodel__alpha':l_ridge_reg}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234)  
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 릿지 학습율:',grid_search.best_params_['basemodel__eta0'])
print('최적 릿지 정규화계수:',grid_search.best_params_['basemodel__alpha'])
R2_scores.append(grid_search.score(X_test,y_test))

최적 선형회귀 학습율: 0.1
최적 선형회귀 스케쥴링: invscaling
최적 라쏘 학습율: 0.1
최적 라쏘 정규화계수: 0.0001
최적 릿지 학습율: 0.1
최적 릿지 정규화계수: 0.001


6. (c) 의사결정트리, 랜덤포레스트, 그래디언트부스팅 기법을 통해, boston 데이터셋에 대해 최적의 계수를 튜닝하고자 합니다. 의사결정트리는 아래 리스트 중 l_decision 리스트를 후보군으로, 트리의 최대깊이를 튜닝합니다. 랜덤포레스트와 그래디언트 부스팅는 각각 l_RF, l_GBT로 시작하는 리스트를 후보군으로 트리 개수와, 피처배깅 하이퍼파라미터를 튜닝합니다. GridSearchCV를 활용하되, 결측치처리 객체, 최소최대정규화 객체를 회귀 전에 수행해야 하며, 4-fold 교차검증을 활용합니다. (k-fold 및 회귀객체의 random_state는 1234로 하세요.) (10pt)

In [56]:
import numpy as np
l_decision=max_depths = [None, 1,2,3,4,5,6]
l_RF_trees=list(np.arange(50,160,20))
l_RF_featurebagging=['sqrt','log2']
l_GBT_trees=list(np.arange(50,160,20))
l_GBT_featurebagging=['sqrt','log2']

답)

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor


pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',DecisionTreeRegressor(random_state=1234))])
param_grid={'basemodel__max_depth':l_decision}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234) 
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 의사결정트리 트리 깊이:',grid_search.best_params_['basemodel__max_depth'])
R2_scores.append(grid_search.score(X_test,y_test))


pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',RandomForestRegressor(random_state=1234))])
param_grid={'basemodel__max_features':l_RF_featurebagging, 'basemodel__n_estimators':l_RF_trees}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234) 
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 랜덤포레스트 트리 개수:',grid_search.best_params_['basemodel__n_estimators'])
print('최적 랜덤포레스트 피처배깅:',grid_search.best_params_['basemodel__max_features'])
R2_scores.append(grid_search.score(X_test,y_test))


pipeline=Pipeline(steps=[('imputer',SimpleImputer()),('scaler',MinMaxScaler()),('basemodel',GradientBoostingRegressor(random_state=1234))])
param_grid={'basemodel__max_features':l_GBT_featurebagging, 'basemodel__n_estimators':l_GBT_trees}
kfold=KFold(n_splits=4, shuffle=True, random_state=1234) 
grid_search=GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train,y_train)
print('최적 GBT 트리 개수:',grid_search.best_params_['basemodel__n_estimators'])
print('최적 GBT 피처배깅:',grid_search.best_params_['basemodel__max_features'])
R2_scores.append(grid_search.score(X_test,y_test))

최적 의사결정트리 트리 깊이: 6
최적 랜덤포레스트 트리 개수: 50
최적 랜덤포레스트 피처배깅: sqrt
최적 GBT 트리 개수: 130
최적 GBT 피처배깅: sqrt


6. (d) 위에서 얻은 최적의 하이퍼파라미터로 훈련된 6가지 모델로 얻은, 테스트 데이터셋에 대한 R2스코어를 출력하세요. ((c)와 (d) 코드를 수정하여 답을 얻어도 됩니다.) (5pt)

답)

In [58]:
R2_scores

[0.7646370952964172,
 0.7646289962595585,
 0.7597872694576642,
 0.8518423744849575,
 0.8912183412671995,
 0.8983824841966818]

7. 랜덤포레스트 모델에서 oob 스코어에 대해 설명하세요. (5pt)

답) 샘플 배깅시에 선택되지 않는 샘플들이 존재하고 이를 out-of-bagging (oob) 샘플이라고 부른다. 따로 테스트 데이터셋으로 측정한 성능이 아니라, 샘플 배깅 시에 자연히 생기는 이 oob 샘플로 측정한 성능을 oob 스코어라고 한다.  